<a href="https://colab.research.google.com/github/Ombiyani/HomologicalQEC/blob/master/513perfectCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import QasmSimulator

In [ ]:
"""
This code implements the 5-qubit [[5,1,3]] quantum error-correcting code,
also known as the perfect code, following the construction described in:

- https://arxiv.org/pdf/1010.3242

It defines logical basis states |0⟩ₗ and |1⟩ₗ, initializes arbitrary or random
superpositions, applies stabilizer measurements, and simulates all single-qubit
Pauli errors to extract the corresponding syndromes.
"""
np.random.seed(42)

def get_logical_ket0():
    return np.array([
        1 / 4, 0, 0, 1 / 4, 0, -1 / 4, 1 / 4, 0,
        0, -1 / 4, -1 / 4, 0, 1 / 4, 0, 0, -1 / 4,
        0, 1 / 4, -1 / 4, 0, -1 / 4, 0, 0, -1 / 4,
        1 / 4, 0, 0, -1 / 4, 0, -1 / 4, -1 / 4, 0
    ])

def get_logical_ket1():
    return np.array([
        0, -1 / 4, -1 / 4, 0, -1 / 4, 0, 0, 1 / 4,
        -1 / 4, 0, 0, -1 / 4, 0, -1 / 4, 1 / 4, 0,
        -1 / 4, 0, 0, 1 / 4, 0, -1 / 4, -1 / 4, 0,
        0, 1 / 4, -1 / 4, 0, 1 / 4, 0, 0, 1 / 4
    ])

def initialize_logical_state(circuit, qubits, alpha=1, beta=0):
    ket0 = get_logical_ket0()
    ket1 = get_logical_ket1()
    state = alpha * ket0 + beta * ket1
    norm = np.linalg.norm(state)
    state /= norm
    init = QuantumCircuit(5)
    init.initialize(state, range(5))
    circuit.append(init.to_instruction(label="|psi_L>"), qubits)

def initialize_random_logical_state(circuit, qubits):
    initialize_logical_state(circuit, qubits, np.random.rand(), np.random.rand())


def apply_stabilizers(circuit, ancillas, data):
    # S0 - IZXXZ
    circuit.cz(ancillas[0], data[1])
    circuit.cx(ancillas[0], data[2])
    circuit.cx(ancillas[0], data[3])
    circuit.cz(ancillas[0], data[4])
    circuit.barrier()
    # S1 - XXZIZ
    circuit.cx(ancillas[1], data[0])
    circuit.cx(ancillas[1], data[1])
    circuit.cz(ancillas[1], data[2])
    circuit.cz(ancillas[1], data[4])
    circuit.barrier()
    # S2 - XZIZX
    circuit.cx(ancillas[2], data[0])
    circuit.cz(ancillas[2], data[1])
    circuit.cz(ancillas[2], data[3])
    circuit.cx(ancillas[2], data[4])
    circuit.barrier()
    # S - ZIZXX
    circuit.cz(ancillas[3], data[0])
    circuit.cz(ancillas[3], data[2])
    circuit.cx(ancillas[3], data[3])
    circuit.cx(ancillas[3], data[4])
    circuit.barrier()

def iterate_single_qubit_errors():
    backend = QasmSimulator()
    errors = ["x", "z", "y"]
    labels = [f"{e.upper()}[{i}]" for e in errors for i in range(5)]
    syndromes = []

    for err_type in errors:
        for i in range(5):
            anc = QuantumRegister(4, name="a")
            data = QuantumRegister(5, name="q")
            creg = ClassicalRegister(4)
            qc = QuantumCircuit(anc, data, creg)

            initialize_random_logical_state(qc, data) # Random superposition
            qc.barrier()

            getattr(qc, err_type)(data[i])
            qc.barrier()

            qc.h(anc)
            qc.barrier()
            apply_stabilizers(qc, anc, data)
            qc.h(anc)
            qc.barrier()
            qc.measure(anc, creg)

            job = backend.run(transpile(qc, backend), shots=128)
            result = job.result().get_counts()
            assert len(result) == 1  # Should only produce one syndrome
            syndrome = list(result.keys())[0]
            syndromes.append(syndrome)

    return pd.DataFrame({"Syndrome": syndromes}, index=pd.Index(labels, name="Single Qubit Error"))

In [ ]:
df = iterate_single_qubit_errors()
print(df)

                   Syndrome
Single Qubit Error         
X[0]                   1000
X[1]                   0101
X[2]                   1010
X[3]                   0100
X[4]                   0011
Z[0]                   0110
Z[1]                   0010
Z[2]                   0001
Z[3]                   1001
Z[4]                   1100
Y[0]                   1110
Y[1]                   0111
Y[2]                   1011
Y[3]                   1101
Y[4]                   1111


In [ ]:
"""
Demonstration of Single-Qubit Error Correction (Syndrome Detection Only):

This function encodes a logical |0⟩ state using a 5-qubit quantum error correction
code, injects a random single-qubit Pauli error (X, Y, or Z), and extracts the
stabilizer syndrome using ancilla qubits. The syndrome indicates the type and
location of the error.

Due to backend limitations (lack of support for `if_test`, `c_if` is deprecated),
the circuit does not apply conditional correction in the same circuit.
Instead, the same error is manually re-applied and corrected in a separate
recovery circuit to simulate the effect of syndrome-based correction.
"""


def apply_correction(circ, pauli, qubit):
    getattr(circ, pauli)(qubit)

def demonstrate_single_error_correction():
    backend = QasmSimulator()
    paulis = ['x', 'y', 'z']
    error_type = np.random.choice(paulis)
    qubit_idx = np.random.randint(0, 5)

    anc = QuantumRegister(4, name="a")
    data = QuantumRegister(5, name="q")
    creg = ClassicalRegister(4)
    qc = QuantumCircuit(anc, data, creg)

    initialize_logical_state(qc, data, alpha=1, beta=0)
    qc.barrier()

    getattr(qc, error_type)(data[qubit_idx])
    qc.barrier()

    qc.h(anc)
    apply_stabilizers(qc, anc, data)
    qc.h(anc)
    qc.measure(anc, creg)

    job = backend.run(transpile(qc, backend), shots=128)
    result = job.result().get_counts()
    syndrome = list(result.keys())[0]

    # Apply correction
    qc_correct = QuantumCircuit(anc, data, creg)
    initialize_logical_state(qc_correct, data, alpha=1, beta=0)
    getattr(qc_correct, error_type)(data[qubit_idx])
    apply_correction(qc_correct, error_type, data[qubit_idx])
    qc_correct.h(anc)
    apply_stabilizers(qc_correct, anc, data)
    qc_correct.h(anc)
    qc_correct.measure(anc, creg)

    job_correct = backend.run(transpile(qc_correct, backend), shots=128)
    result_corrected = job_correct.result().get_counts()
    corrected_syndrome = list(result_corrected.keys())[0]

    print(f"Injected error: {error_type.upper()} on qubit {qubit_idx}")
    print(f"Syndrome before correction: {syndrome}")
    print(f"Syndrome after correction:  {corrected_syndrome}")
demonstrate_single_error_correction()

Injected error: Z on qubit 4
Syndrome before correction: 1100
Syndrome after correction:  0000
